In [ ]:
# Install
# !pip install langchain chromadb anthropic pypdf sentence-transformers

In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
#from langchain.chat_models import ChatAnthropic
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import langchain
langchain.__version__

'0.3.27'

## 1. Load all PDFs from folder

In [4]:
loader = PyPDFDirectoryLoader("/Users/dmitry.bandurin/Dissertation/papers")
documents = loader.load()

In [5]:
documents[0]

Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2021-09-22T05:43:45-04:00', 'moddate': '2021-09-22T05:43:45-04:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '/Users/dmitry.bandurin/Dissertation/papers/Measurement of the photon+b-jet production differential cross section.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='arXiv:1203.5865v2  [hep-ex]  20 Dec 2012FERMILAB-PUB-12-082-E\nMeasurement of the photon+ b-jet production diﬀerential cross section in p¯pcollisions\nat√s= 1.96 TeV\nV.M. Abazov,32B. Abbott,71B.S. Acharya,26M. Adams,47T. Adams,45G.D. Alexeev,32G. Alkhazov,36\nA. Altona,59G. Alverson,58M. Aoki,46A. Askew,45S. Atkins,56K. Augsten,7C. Avila,5F. Badaud,10L. Bagby,46\nB. Baldin,46D.V. Bandurin,45S. Banerjee,26E. Barberis,58P. Baringer,54J. Barreto,2J.F. Bartlett,46\nN. Bartosik,39U. Bassler,15V. Bazterra,47A. Bean,54M. Begalli,2L. Bellantoni

## 2. Smart chunking for articles

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)
chunks = text_splitter.split_documents(documents)

In [7]:
chunks[0]

Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2021-09-22T05:43:45-04:00', 'moddate': '2021-09-22T05:43:45-04:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '/Users/dmitry.bandurin/Dissertation/papers/Measurement of the photon+b-jet production differential cross section.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='arXiv:1203.5865v2  [hep-ex]  20 Dec 2012FERMILAB-PUB-12-082-E\nMeasurement of the photon+ b-jet production diﬀerential cross section in p¯pcollisions\nat√s= 1.96 TeV\nV.M. Abazov,32B. Abbott,71B.S. Acharya,26M. Adams,47T. Adams,45G.D. Alexeev,32G. Alkhazov,36\nA. Altona,59G. Alverson,58M. Aoki,46A. Askew,45S. Atkins,56K. Augsten,7C. Avila,5F. Badaud,10L. Bagby,46\nB. Baldin,46D.V. Bandurin,45S. Banerjee,26E. Barberis,58P. Baringer,54J. Barreto,2J.F. Bartlett,46\nN. Bartosik,39U. Bassler,15V. Bazterra,47A. Bean,54M. Begalli,2L. Bellantoni

In [3]:
# 3. Create embeddings (local, free)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/var/folders/4t/35srr7b93yv4907jj2xsgpzc0000gn/T/ipykernel_46246/2624602784.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/Users/dmitry.bandurin/.pyenv/versions/langchain/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/dmitry.bandurin/.pyenv/versions/3.10.9/envs/langchain/lib/python3.10/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Users/dmitry.bandurin/.pyenv/versions/3.10.9/envs/langchain/lib/python3.10/site-packages/torchvision/image.so
  Expected in: /Users/dmitry.bandur

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4. Store in FAISS

In [8]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [9]:
save_path="./FAISS_storage/faiss_articles_db"
print(f"💿 Saving to {save_path}...")
vectorstore.save_local(save_path)

💿 Saving to ./FAISS_storage/faiss_articles_db...


In [15]:
# 5. Set up Claude
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929",
                   temperature=0)

In [16]:
# 6. Custom prompt for your articles
prompt_template = """You are having a conversation about articles written by the user.
Use the following context from their articles to answer the question.
If you're not sure, say so. Always mention which article(s) you're referencing.

Context:
{context}

Question: {question}

Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [17]:
# 7. Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

In [18]:
# 8. Query!
result = qa_chain({"query": "What did I write about machine learning?"})
print(result['result'])
print("\nSources:", [doc.metadata for doc in result['source_documents']])

Based on the articles you've shared, you wrote about using **Artificial Neural Networks (ANNs)** for particle physics classification. Here are the key points from your article:

**Network Architecture:**
- You used an 11-5-1 architecture (11 input nodes, 5 hidden layer nodes, 1 output node)
- This resulted in 66 independent parameters (Nind = 66)

**Training Process:**
- The network was trained to distinguish signal events (single photons) from background events (π⁰, η, K⁰s mesons)
- Training used target values of 1 for signal and 0 for background
- Weights were initialized randomly and updated by minimizing mean squared error
- The network became stable around epoch 180-200

**Output Interpretation:**
- The network output can be interpreted as a probability (0 to 1)
- Values close to 1 indicate likely signal events
- Values close to 0 indicate likely background events
- You used a "0.5-criterion" threshold for classification

**Performance:**
- You tested the network on different part

In [19]:
result = qa_chain({"query": "what values of effective cross section I have measured in photon+3 jet final state?"})
print(result['result'])
print("\nSources:", [doc.metadata for doc in result['source_documents']])

I don't see any measurements of photon+3 jet final states in the articles you've provided.

The two articles I can reference are:

1. **arXiv:1308.2708v1** - "Measurement of the differential cross section of photon plus jet production in p¯p collisions at √s = 1.96 TeV"

2. **arXiv:1203.5865v2** - "Measurement of the photon+b-jet production differential cross section in p¯p collisions at √s = 1.96 TeV"

Both of these articles focus on **photon + 1 jet** final states (or specifically photon + b-jet in the second case), not photon + 3 jets. The first article measures differential cross sections as a function of photon transverse momentum for different angular configurations of the leading photon and leading jet.

If you have written an article about photon+3 jet measurements, it's not included in the context provided to me.

Sources: [{'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2021-11-24T16:10:18-05:00', 'moddate